In [2]:
import pandas as pd
import numpy as np

## Arrythmia

In [206]:
arrythmia = pd.read_csv('arrhythmia.data', header = None)
arrythmia[1] = arrythmia[1].astype(np.bool)
arrythmia[279] = ~(arrythmia[279] == 1)
arrythmia = (arrythmia.rename({1: 'gender'}, axis = 1)
             .replace({'?': None})
             .astype('float64')
             .astype({'gender':'bool', 279: 'bool'})
             .apply(lambda x: x.fillna(x.mean())))
arrythmia.to_csv('../data/arrythmia.csv', index = False)

,0,gender,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,75,False,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,True
1,56,True,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,True
2,54,False,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,True
3,55,False,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,False
4,75,False,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,True


## Compas

In [24]:
race_cols = ['African_American','Hispanic','Asian','Hispanic','Native_American','Other']

compas = (pd.read_csv('compass.csv')
 .assign(Caucasian = lambda df: df[race_cols].sum(axis=1) == 0,
        Y= lambda df: df.Two_yr_Recidivism)
 .drop('Two_yr_Recidivism', axis = 1)
 .query('Caucasian or African_American')
 .drop(race_cols, axis = 1)
 .rename({'Female':'gender', 'Caucasian': 'race'}, axis=1)
 .astype({'Y': 'bool'}))
compas.to_csv('../data/compas.csv', index=False)

## German

In [283]:
german = (pd.read_csv('german.data', header = None, delimiter = ' ')
          .replace(['A91','A93','A94'],True)
          .replace(['A92','A95'], False)
          .rename({8:'gender'},axis=1)
          .assign(foreign = lambda df: df[19] == 'A201',
                 Y = lambda df: df[20]-1)
          .drop([19,20], axis=1)
          .astype({'Y':'bool'})
          .to_csv('../data/german.csv', index = False))

## Drug

In [286]:
drug = pd.read_csv('drug_consumption.data', header = None)
drug['race'] = drug[5] == -0.31685
drug['Y'] = drug.loc[:,29] == 'CL0'
#drug.loc[:,31-18::]
drug = drug.drop(np.arange(31-18,32), axis = 1)
drug = drug.drop([0,5], axis = 1)
drug.to_csv('../data/drug.csv', index = False)

## Default

In [ ]:
default = (pd.read_excel('default of credit card clients.xls')
           .drop(0)
           .drop('Unnamed: 0', axis=1)
           .astype('float64')
           .assign(X2= lambda df: df.X2 == 1)
           .astype({'Y': 'bool', 
                   'X2':'bool'}))


default.to_csv('../data/default.csv', index = False)

## CART Benchmarking

In [337]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score

X = drug.drop('Y', axis = 1).to_numpy()
Y = drug['Y'].to_numpy()
g = drug['race'].to_numpy()

min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.7729342564448948  with fairness 0.0
Best Fairness:  0.7729342564448948  with fairness 0.0


In [327]:
arrythmia.head()

,0,gender,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,75.0,False,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,True
1,56.0,True,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,True
2,54.0,False,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,True
3,55.0,False,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,False
4,75.0,False,190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,True


In [338]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score

X = arrythmia.drop(279, axis = 1).to_numpy()
Y = arrythmia[279].to_numpy()
g = arrythmia['gender'].to_numpy()

min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.6745893719806764  with fairness 0.2562670662670663
Best Fairness:  0.6745893719806764  with fairness 0.2562670662670663


In [3]:
def getFold(X,Y,group, indices):
    X_train = np.delete(X, indices, axis=0)
    Y_train = np.delete(Y, indices)
    g_train = np.delete(group, indices)

    X_test = X[indices,:]
    Y_test = Y[indices]
    g_test = group[indices]
    
    return X_train, Y_train, g_train, X_test, Y_test, g_test

def fairCrossValScore(model, X, Y, g, cv=10, hp=10):
    break_points = np.floor(np.arange(0,1+1/cv, 1/cv) * X.shape[0]).astype(np.int)
    acc = []
    eqGap = []
    eqGapPos = []
    eqGapNeg = []

    for i in range(cv):
        X_tr, Y_tr, g_tr, X_t, Y_t, g_t = getFold(X, Y, g, np.arange(break_points[i], break_points[i+1]))
        model.fit(X_tr, Y_tr)
        preds = model.predict(X_t)
        accuracy = (preds == Y_t).mean()
        EqOfOpGap = abs((preds[g_t] == Y_t[g_t]).mean() - (preds[~g_t] == Y_t[~g_t]).mean())
        EqOfOpGapPos = abs((preds[g_t & Y_t] == 1).mean() - (preds[~g_t & Y_t] == 1).mean())
        EqOfOpGapNeg = abs((preds[g_t & ~Y_t] == 0).mean() - (preds[~g_t & ~Y_t] == 0).mean())

        acc.append(accuracy)
        eqGap.append(EqOfOpGap)
        eqGapPos.append(EqOfOpGapPos)
        eqGapNeg.append(EqOfOpGapNeg)

    return np.array(acc), np.array(eqGap),np.array(eqGapPos),np.array(eqGapNeg)


In [308]:
cv=10
break_points = np.floor(np.arange(0,1+1/cv, 1/cv) * X.shape[0]).astype(np.int)
break_points

array([   0,  188,  377,  565,  754,  942, 1131, 1319, 1508, 1696, 1885])

In [299]:
cross_val_score?

In [292]:
min_samples[np.argmax(results)]

200

In [46]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score
adult = pd.get_dummies(pd.read_csv('adult.csv'))

X = adult.drop('Y', axis = 1).to_numpy()
Y = adult['Y'].to_numpy()
g = adult['gender'].to_numpy()

min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []
fair_pos = []
fair_pos_sd = []
fair_neg = []
fair_neg_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())
    fair_pos.append(scores[2].mean())
    fair_pos_sd.append(scores[2].std())
    fair_neg.append(scores[3].mean())
    fair_neg_sd.append(scores[3].std())



best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.8551027403460625  with fairness 0.10632812445973591
Best Fairness:  0.8551027403460625  with fairness 0.10632812445973591


In [59]:
adult['Y'].value_counts(True)

False    0.75919
True     0.24081
Name: Y, dtype: float64

In [47]:
adult_bench_cart = (pd.DataFrame({'algo': 'cart', 'Accuracy': acc, 
                                  'HamGap': fair,
                                 'HamGapPos': fair_pos,
                                  'HamGapPos_sd': fair_pos_sd,
                                  'HamGapNeg_sd': fair_neg_sd,
                                 'HamGapNeg': fair_neg})
                      .sort_values('HamGap')
                     .assign(cumAccHamGap = lambda df: df.Accuracy.cummax())
                     .sort_values('HamGapPos')
                     .assign(cumAccHamGapPos = lambda df: df.Accuracy.cummax())
                      .sort_values('HamGapNeg')
                     .assign(cumAccHamGapNeg = lambda df: df.Accuracy.cummax()))

adult_bench_cart

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
3,cart,0.853199,0.107904,0.172683,0.050757,0.005556,0.068547,0.855103,0.855103,0.853199
2,cart,0.854611,0.107747,0.155069,0.050122,0.005436,0.068567,0.855103,0.854611,0.854611
0,cart,0.853659,0.112783,0.084271,0.043122,0.008630,0.070636,0.855103,0.853659,0.854611
1,cart,0.855103,0.106328,0.158982,0.051180,0.005736,0.072356,0.855103,0.855103,0.855103
4,cart,0.850895,0.109834,0.180885,0.057208,0.007922,0.074900,0.855103,0.855103,0.855103
6,cart,0.841989,0.110386,0.169890,0.050333,0.016107,0.075630,0.855103,0.855103,0.855103
5,cart,0.845183,0.110196,0.169443,0.040516,0.010040,0.075847,0.855103,0.855103,0.855103
7,cart,0.839317,0.116977,0.126175,0.047782,0.015940,0.087280,0.855103,0.853659,0.855103
10,cart,0.824361,0.131577,0.175668,0.039999,0.007250,0.094689,0.855103,0.855103,0.855103
8,cart,0.836062,0.119484,0.183161,0.033634,0.010087,0.096457,0.855103,0.855103,0.855103


In [48]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import cross_val_score
adult = pd.get_dummies(pd.read_csv('adult.csv'))

X = adult.drop('Y', axis = 1).to_numpy()
Y = adult['Y'].to_numpy()
g = adult['gender'].to_numpy()

Cs = [0.01, 0.1, 1, 3, 10]
acc = []
acc_sd = []
fair = []
fair_sd = []
fair_pos = []
fair_pos_sd = []
fair_neg = []
fair_neg_sd = []

for C in Cs:
    cart = LogisticRegression(C = C, max_iter = 1000)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())
    fair_pos.append(scores[2].mean())
    fair_pos_sd.append(scores[2].std())
    fair_neg.append(scores[3].mean())
    fair_neg_sd.append(scores[3].std())



best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.8009889774358611  with fairness 0.1499439330620607
Best Fairness:  0.8009889774358611  with fairness 0.1499439330620607


In [49]:
adult_bench_lr = (pd.DataFrame({'algo': 'logreg', 'Accuracy': acc, 
                                  'HamGap': fair,
                                 'HamGapPos': fair_pos,
                                  'HamGapPos_sd': fair_pos_sd,
                                  'HamGapNeg_sd': fair_neg_sd,
                                 'HamGapNeg': fair_neg})
                      .sort_values('HamGap')
                     .assign(cumAccHamGap = lambda df: df.Accuracy.cummax())
                     .sort_values('HamGapPos')
                     .assign(cumAccHamGapPos = lambda df: df.Accuracy.cummax())
                      .sort_values('HamGapNeg')
                     .assign(cumAccHamGapNeg = lambda df: df.Accuracy.cummax()))

adult_bench_lr

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
2,logreg,0.797918,0.146074,0.035770,0.032350,0.005252,0.009835,0.797918,0.797918,0.797918
0,logreg,0.798041,0.149509,0.044738,0.033824,0.022348,0.017668,0.798041,0.798041,0.798041
3,logreg,0.797242,0.150878,0.042077,0.041377,0.036633,0.026882,0.800989,0.797918,0.798041
1,logreg,0.796536,0.153834,0.044171,0.033340,0.036228,0.027702,0.800989,0.797918,0.798041
4,logreg,0.800989,0.149944,0.070642,0.079575,0.043212,0.037690,0.800989,0.800989,0.800989


In [55]:
adult_bench_lr[adult_bench_lr.HamGapNeg == adult_bench_lr.HamGapNeg.min()]

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
2,logreg,0.797918,0.146074,0.03577,0.03235,0.005252,0.009835,0.797918,0.797918,0.797918


In [16]:
adult_bench = adult_bench_cart.append(adult_bench_lr) #.to_csv('benchmark_adult_heo.csv')

In [53]:
adult_bench_cart[adult_bench_cart.HamGapNeg == adult_bench_cart.HamGapNeg.min()]

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
3,cart,0.853199,0.107904,0.172683,0.050757,0.005556,0.068547,0.855103,0.855103,0.853199


In [28]:
adult_bench = pd.DataFrame({'algo': 'logReg', 'Accuracy': acc, 'EqOfOp': fair})
bestOfAct = np.argmax(adult_bench.Accuracy)
bestOfFair = np.argmin(adult_bench.EqOfOp)
adult_bench.iloc[[bestOfAct, bestOfFair],:]

,algo,Accuracy,EqOfOp
4,logReg,0.800989,0.070642
2,logReg,0.797918,0.035770


In [22]:
adult_full_bench = adult_bench.append(adult_bench_cart)
(adult_full_bench.groupby(['algo'])
 .sort_values('Accuracy', ascending = True)
 .assign(cumAcc=lambda df: df.Accuracy.cummax()))

AttributeError: 'DataFrameGroupBy' object has no attribute 'assign'

In [415]:
default = (pd.read_excel('default of credit card clients.xls')
           .drop(0)
           .drop('Unnamed: 0', axis=1)
           .astype('float64')
           .assign(X2= lambda df: df.X2 == 1)
           .astype({'Y': 'bool', 
                   'X2':'bool'}))


default.to_csv('../data/default.csv', index = False)

In [4]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score
default = pd.get_dummies(pd.read_csv('../data/default.csv'))

X = default.drop('Y', axis = 1).to_numpy()
Y = default['Y'].to_numpy()
g = default['X2'].to_numpy()

min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []
fair_pos = []
fair_pos_sd = []
fair_neg = []
fair_neg_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())
    fair_pos.append(scores[2].mean())
    fair_pos_sd.append(scores[2].std())
    fair_neg.append(scores[3].mean())
    fair_neg_sd.append(scores[3].std())


best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.8205666666666668  with fairness 0.02347642056874214
Best Fairness:  0.8205666666666668  with fairness 0.02347642056874214


In [5]:
default_bench_cart = (pd.DataFrame({'algo': 'cart', 'Accuracy': acc, 
                                  'HamGap': fair,
                                 'HamGapPos': fair_pos,
                                  'HamGapPos_sd': fair_pos_sd,
                                  'HamGapNeg_sd': fair_neg_sd,
                                    'HamGapNeg': fair_neg})
                      .sort_values('HamGap')
                     .assign(cumAccHamGap = lambda df: df.Accuracy.cummax())
                     .sort_values('HamGapPos')
                     .assign(cumAccHamGapPos = lambda df: df.Accuracy.cummax())
                      .sort_values('HamGapNeg')
                     .assign(cumAccHamGapNeg = lambda df: df.Accuracy.cummax()))

default_bench_cart

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
10,cart,0.819600,0.028468,0.025732,0.018745,0.006620,0.010651,0.820567,0.819600,0.819600
9,cart,0.819133,0.029308,0.026249,0.019467,0.007387,0.011657,0.820567,0.819600,0.819600
5,cart,0.819800,0.023096,0.028369,0.026554,0.006522,0.012119,0.819800,0.819800,0.819800
6,cart,0.819800,0.023096,0.028369,0.026554,0.006522,0.012119,0.819800,0.819800,0.819800
7,cart,0.819800,0.023096,0.028369,0.026554,0.006522,0.012119,0.819800,0.819800,0.819800
8,cart,0.820567,0.023476,0.029909,0.027369,0.006739,0.013147,0.820567,0.820567,0.820567
1,cart,0.818167,0.026406,0.037382,0.026742,0.009438,0.019335,0.820567,0.820567,0.820567
0,cart,0.812433,0.031905,0.026186,0.024778,0.014027,0.019782,0.820567,0.819600,0.820567
4,cart,0.819667,0.026391,0.042691,0.028413,0.011836,0.020493,0.820567,0.820567,0.820567
2,cart,0.819000,0.025132,0.048624,0.040502,0.013191,0.021562,0.820567,0.820567,0.820567


In [6]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import cross_val_score

default = pd.get_dummies(pd.read_csv('../data/default.csv'))

X = default.drop('Y', axis = 1).to_numpy()
Y = default['Y'].to_numpy()
g = default['X2'].to_numpy()

Cs = [0.01, 0.1, 1, 3, 10]
acc = []
acc_sd = []
fair = []
fair_sd = []
fair_pos = []
fair_pos_sd = []
fair_neg = []
fair_neg_sd = []

for C in Cs:
    cart = LogisticRegression(C = C, max_iter = 1000)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())
    fair_pos.append(scores[2].mean())
    fair_pos_sd.append(scores[2].std())
    fair_neg.append(scores[3].mean())
    fair_neg_sd.append(scores[3].std())


best_acc = np.argmax(acc)
best_fair = np.argmin(fair)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.7787666666666667  with fairness 0.03389524383468766
Best Fairness:  0.7787666666666667  with fairness 0.03389524383468766


In [7]:
default_bench_lr = (pd.DataFrame({'algo': 'logreg', 'Accuracy': acc, 
                                  'HamGap': fair,
                                 'HamGapPos': fair_pos,
                                  'HamGapPos_sd': fair_pos_sd,
                                  'HamGapNeg_sd': fair_neg_sd,
                                 'HamGapNeg': fair_neg})
                      .sort_values('HamGap')
                     .assign(cumAccHamGap = lambda df: df.Accuracy.cummax())
                     .sort_values('HamGapPos')
                     .assign(cumAccHamGapPos = lambda df: df.Accuracy.cummax())
                      .sort_values('HamGapNeg')
                     .assign(cumAccHamGapNeg = lambda df: df.Accuracy.cummax()))

default_bench_lr

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
0,logreg,0.778767,0.033895,0.0,0.0,0.00022,0.000073,0.778767,0.778767,0.778767
1,logreg,0.778767,0.033895,0.0,0.0,0.00022,0.000073,0.778767,0.778767,0.778767
2,logreg,0.778767,0.033895,0.0,0.0,0.00022,0.000073,0.778767,0.778767,0.778767
3,logreg,0.778767,0.033895,0.0,0.0,0.00022,0.000073,0.778767,0.778767,0.778767
4,logreg,0.778767,0.033895,0.0,0.0,0.00022,0.000073,0.778767,0.778767,0.778767


In [22]:
default_bench_lr.append(default_bench_cart)

In [12]:
default_bench = default_bench_lr.append(default_bench_cart)

In [86]:
default['Y'].value_counts(True)

False    0.7788
True     0.2212
Name: Y, dtype: float64

In [ ]:
default_bench_lr = pd.DataFrame({'algo': 'logreg', 'Accuracy': acc, 'EqOfOp': fair})
bestOfAct = np.argmax(default_bench_lr.Accuracy)
bestOfFair = np.argmin(default_bench_lr.EqOfOp)
default_bench_lr.iloc[[bestOfAct, bestOfFair],:]

In [419]:
min_samples[np.argmax(acc)]

600

In [360]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score

X = default.drop('Y', axis = 1).to_numpy()
Y = default['Y'].to_numpy()
g = default['X2'].to_numpy()
arrythmia = pd.read_csv('arrhythmia.data', header = None)
arrythmia[1] = arrythmia[1].astype(np.bool)
arrythmia[279] = ~(arrythmia[279] == 1)
arrythmia = (arrythmia.rename({1: 'gender'}, axis = 1)
             .replace({'?': None})
             .astype('float64')
             .astype({'gender':'bool', 279: 'bool'})
             .apply(lambda x: x.fillna(x.mean())))
arrythmia.to_csv('../data/arrythmia.csv', index = False)
min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.8205666666666668  with fairness 0.02990852760752783
Best Fairness:  0.8205666666666668  with fairness 0.02990852760752783


In [66]:
compas['Y'].value_counts(True)

False    0.529557
True     0.470443
Name: Y, dtype: float64

In [32]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score

X = compas.drop('Y', axis = 1).to_numpy()
Y = compas['Y'].to_numpy()
g = compas['race'].to_numpy()

min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []
fair_pos = []
fair_pos_sd = []
fair_neg = []
fair_neg_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())
    fair_pos.append(scores[2].mean())
    fair_pos_sd.append(scores[2].std())
    fair_neg.append(scores[3].mean())
    fair_neg_sd.append(scores[3].std())


best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.6813211574952562  with fairness 0.04473495491166266
Best Fairness:  0.6813211574952562  with fairness 0.04473495491166266


In [33]:
compas_bench_cart = (pd.DataFrame({'algo': 'cart', 'Accuracy': acc, 
                                  'HamGap': fair,
                                 'HamGapPos': fair_pos,
                                  'HamGapPos_sd': fair_pos_sd,
                                  'HamGapNeg_sd': fair_neg_sd,
                                 'HamGapNeg': fair_neg})
                      .sort_values('HamGap')
                     .assign(cumAccHamGap = lambda df: df.Accuracy.cummax())
                     .sort_values('HamGapPos')
                     .assign(cumAccHamGapPos = lambda df: df.Accuracy.cummax())
                      .sort_values('HamGapNeg')
                     .assign(cumAccHamGapNeg = lambda df: df.Accuracy.cummax()))

compas_bench_cart

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
5,cart,0.668249,0.050023,0.247001,0.049113,0.053745,0.168197,0.681321,0.668249,0.668249
6,cart,0.668249,0.050023,0.247001,0.049113,0.053745,0.168197,0.681321,0.668249,0.668249
4,cart,0.666733,0.054403,0.246561,0.063200,0.052970,0.179222,0.681321,0.666733,0.668249
3,cart,0.666542,0.046774,0.244025,0.060772,0.052567,0.180184,0.681321,0.666542,0.668249
2,cart,0.664836,0.049048,0.248195,0.075949,0.038602,0.189594,0.681321,0.668249,0.668249
7,cart,0.659348,0.046984,0.213696,0.054721,0.060491,0.192615,0.681321,0.659348,0.668249
8,cart,0.659348,0.046984,0.213696,0.054721,0.060491,0.192615,0.681321,0.659348,0.668249
9,cart,0.659348,0.046984,0.213696,0.054721,0.060491,0.192615,0.681321,0.659348,0.668249
10,cart,0.658209,0.050499,0.213399,0.061110,0.056026,0.202465,0.681321,0.658209,0.668249
1,cart,0.672986,0.040578,0.274395,0.055779,0.059009,0.213905,0.672986,0.681321,0.672986


In [30]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import cross_val_score

compas = pd.read_csv('../data/compas.csv')
X = compas.drop('Y', axis = 1).to_numpy()
Y = compas['Y'].to_numpy()
g = compas['race'].to_numpy()

Cs = [0.01, 0.1, 1, 3, 10]
acc = []
acc_sd = []
fair = []
fair_sd = []

Cs = [0.01, 0.1, 1, 3, 10]
acc = []
acc_sd = []
fair = []
fair_sd = []
fair_pos = []
fair_pos_sd = []
fair_neg = []
fair_neg_sd = []

for C in Cs:
    cart = LogisticRegression(C = C, max_iter = 1000)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())
    fair_pos.append(scores[2].mean())
    fair_pos_sd.append(scores[2].std())
    fair_neg.append(scores[3].mean())
    fair_neg_sd.append(scores[3].std())


best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.6809466821919383  with fairness 0.047935770818011994
Best Fairness:  0.6809466821919383  with fairness 0.047935770818011994


In [31]:
compas_bench_lr = (pd.DataFrame({'algo': 'logreg', 'Accuracy': acc, 
                                  'HamGap': fair,
                                 'HamGapPos': fair_pos,
                                  'HamGapPos_sd': fair_pos_sd,
                                  'HamGapNeg_sd': fair_neg_sd,
                                 'HamGapNeg': fair_neg})
                      .sort_values('HamGap')
                     .assign(cumAccHamGap = lambda df: df.Accuracy.cummax())
                     .sort_values('HamGapPos')
                     .assign(cumAccHamGapPos = lambda df: df.Accuracy.cummax())
                      .sort_values('HamGapNeg')
                     .assign(cumAccHamGapNeg = lambda df: df.Accuracy.cummax()))

compas_bench_lr

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
3,logreg,0.675450,0.043449,0.271534,0.064071,0.045807,0.191238,0.675450,0.680947,0.675450
4,logreg,0.675450,0.043449,0.271534,0.064071,0.045807,0.191238,0.675450,0.680947,0.675450
2,logreg,0.675261,0.043707,0.271534,0.064071,0.045988,0.191928,0.675450,0.680947,0.675450
1,logreg,0.675640,0.046300,0.271550,0.065143,0.047542,0.195754,0.675640,0.680947,0.675640
0,logreg,0.680947,0.047936,0.271292,0.075766,0.049112,0.205560,0.680947,0.680947,0.680947


In [35]:
compas_bench = compas_bench_lr.append(compas_bench_cart)#.to_csv("benchmark_compas_heo.csv")

In [38]:
#compas_bench_cart = compas_bench[compas_bench.algo == 'cart']
#compas_bench_cart #[compas_bench_cart.Accuracy == compas_bench_cart.Accuracy.max()]
compas_bench_cart[compas_bench_cart.Accuracy == compas_bench_cart.Accuracy.max()]

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
0,cart,0.681321,0.044735,0.255683,0.061892,0.052882,0.214734,0.681321,0.681321,0.681321


In [23]:
default_bench[default_bench.accuracy == default_bench.accuracy.max()]

AttributeError: 'DataFrame' object has no attribute 'accuracy'

In [64]:
default_bench

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
0,logreg,0.778767,0.033895,0.000000,0.000000,0.000220,0.000073,0.778767,0.778767,0.778767
1,logreg,0.778767,0.033895,0.000000,0.000000,0.000220,0.000073,0.778767,0.778767,0.778767
2,logreg,0.778767,0.033895,0.000000,0.000000,0.000220,0.000073,0.778767,0.778767,0.778767
3,logreg,0.778767,0.033895,0.000000,0.000000,0.000220,0.000073,0.778767,0.778767,0.778767
4,logreg,0.778767,0.033895,0.000000,0.000000,0.000220,0.000073,0.778767,0.778767,0.778767
10,cart,0.819600,0.028468,0.025732,0.018745,0.006620,0.010651,0.820567,0.819600,0.819600
9,cart,0.819133,0.029308,0.026249,0.019467,0.007387,0.011657,0.820567,0.819600,0.819600
5,cart,0.819800,0.023096,0.028369,0.026554,0.006522,0.012119,0.819800,0.819800,0.819800
6,cart,0.819800,0.023096,0.028369,0.026554,0.006522,0.012119,0.819800,0.819800,0.819800
7,cart,0.819800,0.023096,0.028369,0.026554,0.006522,0.012119,0.819800,0.819800,0.819800


In [66]:
default_lr = default_bench[default_bench.algo == 'logreg']
default_lr[default_lr.HamGapNeg == default_lr.HamGapNeg.min()]

,algo,Accuracy,HamGap,HamGapPos,HamGapPos_sd,HamGapNeg_sd,HamGapNeg,cumAccHamGap,cumAccHamGapPos,cumAccHamGapNeg
0,logreg,0.778767,0.033895,0.0,0.0,0.00022,0.000073,0.778767,0.778767,0.778767
1,logreg,0.778767,0.033895,0.0,0.0,0.00022,0.000073,0.778767,0.778767,0.778767
2,logreg,0.778767,0.033895,0.0,0.0,0.00022,0.000073,0.778767,0.778767,0.778767
3,logreg,0.778767,0.033895,0.0,0.0,0.00022,0.000073,0.778767,0.778767,0.778767
4,logreg,0.778767,0.033895,0.0,0.0,0.00022,0.000073,0.778767,0.778767,0.778767
